Dataset Preprocessing

In [1]:
!pip install datasets
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
import pandas as pd
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
dataset = load_dataset("empathetic_dialogues")
print(dataset)

README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})


In [8]:
df_train = dataset['train'].to_pandas()
df_train.head()

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,


In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    #Convert text to lowercase
    text = text.lower()

    #Remove URLs
    text = re.sub(r'http\S+', '', text)

    #Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Remove numeric characters
    text = re.sub(r'\d+', '', text)

    #Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    #Tokenization
    tokens = word_tokenize(text)

    #Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    #Rejoin tokens
    cleaned_text = ' '.join(tokens)

    return cleaned_text

#Apply preprocessing on utterances
df_train['cleaned_utterance'] = df_train['utterance'].apply(preprocess_text)
df_train[['utterance', 'cleaned_utterance']].head()

,utterance,cleaned_utterance
0,I remember going to see the fireworks with my ...,remember going see fireworks best friend first...
1,Was this a friend you were in love with_comma_...,friend love withcomma best friend
2,This was a best friend. I miss her.,best friend miss
3,Where has she gone?,gone
4,We no longer talk.,longer talk


In [12]:
df_train.to_csv('empathetic_dialogues_cleaned.csv', index=False)

In [14]:
!pip install transformers datasets evaluate -q
!pip install -q accelerate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.5 MB/s eta 0:00:00


tokenizing and training the pretrained model

In [22]:
import pandas as pd

df = pd.read_csv("empathetic_dialogues_cleaned.csv")

#Map emotions to labels
label2id = {label: idx for idx, label in enumerate(df['context'].unique())}
id2label = {idx: label for label, idx in label2id.items()}
df['label'] = df['context'].map(label2id)

In [24]:
df = df[['cleaned_utterance', 'context']].dropna()
df['cleaned_utterance'] = df['cleaned_utterance'].astype(str)
df['context'] = df['context'].astype(str)

#Rebuild labels
label2id = {label: idx for idx, label in enumerate(df['context'].unique())}
id2label = {idx: label for label, idx in label2id.items()}
df['label'] = df['context'].map(label2id)

In [27]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[['cleaned_utterance', 'label']])

In [30]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

#Convert DataFrame
dataset = Dataset.from_pandas(df[['cleaned_utterance', 'label']])

#Tokenization function
def tokenize_function(example):
    return tokenizer(example["cleaned_utterance"], padding="max_length", truncation=True)

#Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

#Split into train and test sets
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)


Map:   0%|          | 0/76092 [00:00<?, ? examples/s]

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


training

In [43]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    warmup_steps=500,
    gradient_accumulation_steps=2,
    fp16=True,
)

In [44]:
#AdamW with custom param groups
from torch.optim import AdamW

def get_optimizer(model):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.01,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    return AdamW(optimizer_grouped_parameters, lr=2e-5)

In [45]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

In [46]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def create_optimizer(self):
        self.optimizer = get_optimizer(self.model)

In [47]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-47-f6376ce4ecd8>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.537600,2.617338,0.273868,0.268487
2,2.377600,2.587096,0.281687,0.278044
3,2.244800,2.596026,0.286747,0.281311
4,2.097200,2.635697,0.287075,0.281777
5,1.927300,2.717148,0.284644,0.281527
6,1.758400,2.795557,0.280833,0.278109
7,1.592400,2.911699,0.278928,0.277784
8,1.459400,3.047865,0.270057,0.267832
9,1.307500,3.137552,0.269794,0.268938
10,1.148800,3.311347,0.270583,0.269749


TrainOutput(global_step=38040, training_loss=1.2495798816440233, metrics={'train_runtime': 6270.5705, 'train_samples_per_second': 194.155, 'train_steps_per_second': 6.066, 'total_flos': 3.202510848663552e+17, 'train_loss': 1.2495798816440233, 'epoch': 19.98975032851511})

In [48]:
trainer.save_model("empathetic-transformer")
tokenizer.save_pretrained("empathetic-transformer")

('empathetic-transformer/tokenizer_config.json',
 'empathetic-transformer/special_tokens_map.json',
 'empathetic-transformer/vocab.json',
 'empathetic-transformer/merges.txt',
 'empathetic-transformer/added_tokens.json',
 'empathetic-transformer/tokenizer.json')

In [49]:
pip install fastapi uvicorn google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00


In [60]:
import os
from getpass import getpass
import google.generativeai as genai
from transformers import pipeline

#Gemini API key from environment
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    gemini_api_key = getpass("Enter your Gemini API key: ")

#Configure Gemini API
genai.configure(api_key=gemini_api_key)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

from fastapi import FastAPI, WebSocket, HTTPException
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI(title="Empathetic Adaptive Response Engine")

@app.get("/")
async def read_root():
    return {"message": "Welcome to the Empathetic Adaptive Response Engine! Visit /docs for API details."}

class ChatRequest(BaseModel):
    user_message: str
    emotion: str

#For automatic emotion detection
class AutoChatRequest(BaseModel):
    user_message: str

emotion_classifier = pipeline(
    "text-classification",
    model="empathetic-transformer",
    tokenizer="empathetic-transformer",
    return_all_scores=False
)

def build_prompt(user_message: str, emotion: str) -> str:
    prompt = f"""
You are an empathetic and supportive AI assistant.
The user is currently feeling {emotion}.
User says: "{user_message}"
Generate a compassionate, thoughtful, and contextually appropriate response.
"""
    return prompt

@app.post("/auto_generate_response", summary="Auto Generate a Response",
          description="Automatically detects the user's emotion from their message and returns an empathetic response.")
async def auto_generate_response(chat_req: AutoChatRequest):
    try:
        classifier_output = emotion_classifier(chat_req.user_message)
        predicted_emotion = classifier_output[0]["label"]
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error in emotion classification: {e}")

    prompt = build_prompt(chat_req.user_message, predicted_emotion)
    try:
        response = gemini_model.generate_content(prompt)
        response_text = response.text
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error with Gemini API: {e}")

    return {"detected_emotion": predicted_emotion, "response": response_text}

@app.post("/generate_response", summary="Generate a Response (Manual)",
          description="Generates an empathetic response based on user input and provided emotion (manual input).")
async def generate_response(chat_req: ChatRequest):
    prompt = build_prompt(chat_req.user_message, chat_req.emotion)
    try:
        response = gemini_model.generate_content(prompt)
        response_text = response.text
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error with Gemini API: {e}")
    return {"response": response_text}

#WebSocket endpoint
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        try:
            data = await websocket.receive_json()
            user_message = data.get("message")
            if not user_message:
                await websocket.send_json({"error": "Invalid input, missing 'message'"})
                continue
            try:
                classifier_output = emotion_classifier(user_message)
                predicted_emotion = classifier_output[0]["label"]
            except Exception as e:
                await websocket.send_json({"error": f"Error in emotion classification: {str(e)}"})
                continue

            #Build and send prompt to gemini
            prompt = build_prompt(user_message, predicted_emotion)
            try:
                response = gemini_model.generate_content(prompt)
                response_text = response.text
            except Exception as e:
                await websocket.send_json({"error": f"Error with Gemini API: {str(e)}"})
                continue
            await websocket.send_json({"detected_emotion": predicted_emotion, "response": response_text})
        except Exception as e:
            await websocket.close()
            break

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-10' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

Enter your Gemini API key: ··········


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [61]:
!pip install pyngrok
from pyngrok import ngrok
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://6dba-34-143-144-253.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2255]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     172.98.143.216:0 - "GET / HTTP/1.1" 200 OK
INFO:     172.98.143.216:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     172.98.143.216:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     172.98.143.216:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     172.98.143.216:0 - "POST /auto_generate_response HTTP/1.1" 200 OK
INFO:     172.98.143.216:0 - "POST /auto_generate_response HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2255]
